<a href="https://colab.research.google.com/github/R0N3ldrt/Thesis/blob/main/NN_predict_v40.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Traininig NN Spectrum

# Loading Necesary Libraries

In [1]:
# Importing necesary libraries
# Libraries for correct code execution 

import os, time, joblib
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import math
import pickle
import random
import csv
import re
import array
import seaborn as sns
from tqdm import tqdm
import tensorflow as tf
import os.path

from scipy import stats
from functools import reduce
from random import random, gauss
from math import modf, pi, cos, sin, sqrt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import r2_score
from plotly.subplots import make_subplots
from scipy.signal import savgol_filter
from scipy.stats.stats import pearsonr
from scipy.stats import spearmanr

import scipy.stats as st
import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots

from keras.models import Sequential
from keras.layers import Dense,Conv2D, Flatten
from keras.wrappers.scikit_learn import KerasRegressor
from keras.callbacks import EarlyStopping
from sklearn.mixture import GaussianMixture
import os, time, math, csv, joblib
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.mixture import GaussianMixture

import os, time, math, csv, joblib
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.mixture import GaussianMixture

import warnings
warnings.filterwarnings("ignore")

sns.set_theme()

# Load Enviroment

In [2]:
# BLOQUE PARA USAR DESDE COLAB

# Google drive loading as work station for local-usage of the files.
from google.colab import drive
drive.mount('/content/gdrive',force_remount= True)

#-----------------------------------------------------------------------------

Mounted at /content/gdrive


In [3]:
# Lista para cambiar los paths rapido.
workers = ["Ronald", "Local"]

# Change the number to change the paths.
worker = workers[0]

if worker == "Ronald":
  path = "/content/gdrive/MyDrive/Thesis_Workstation/ANN_dataset"
else: path = os.getcwd()

# Get working_df

## Aggregator

In [4]:
def prime_number_finder(stop_gap, total_num_of_data):
  prime_nums = []
  i=2
  while i <= stop_gap:
    if (total_num_of_data % i==0):
      prime_nums.append(i)
      break
    i+=1
  return prime_nums

In [5]:
def aggreagator_v2(df, stop_gap=50):
  total_num_of_data = df.shape[1]
  arr_prime = prime_number_finder(stop_gap, total_num_of_data)
  #agg_num = np.max(arr_prime)
  agg_num = arr_prime[0]

  # Creating new df of aggregate values
  agg_df = pd.DataFrame()

  mid_point = df.shape[1]/2

  agg=0
  loop_cnt=0
  init_column_cnt = 5
  while agg<=total_num_of_data:
    loop_cnt+=1
    # Obtain current last columns stop
    agg=agg_num+init_column_cnt
    # Select working columns
    new_df = df[df.columns[init_column_cnt:agg]]

    init_column_cnt += (agg_num)

    headers = list(new_df.columns.values) 
    if loop_cnt <= mid_point:
      # Get average of freq values for new header
      new_header_name  = headers[-1]
    else:
      new_header_name = headers[0]
    
    # Add aggragated values to new df
    agg_df[new_header_name] = new_df.mean(axis = 1)
  return agg_df

## Find cutoff index

In [6]:
def remove_tail(df, cut_val, sample_id):
  columns_selected = []
  old_val = 0
  delta = 0
  mid_point = int(df.shape[1]/2)
  cols_headers  = list(df.columns.values)

  for i in range(0, df.shape[1]):
    if i == 0:
      cell_val = df.iloc[[0], i]
      old_val = cell_val[sample_id]
    else:
      cell_val = df.iloc[[0], i]
      val = cell_val[sample_id]
      delta = abs(old_val-val)
      old_val = val
      if delta > cut_val:
        if i <= mid_point:
          col_name_selected = cols_headers[i+2]
        else:
          col_name_selected = cols_headers[i-2]
          col_name_selected = round(float(col_name_selected), 5)
        columns_selected.append(col_name_selected)

  return columns_selected

In [7]:
# Removing tails conducting multiples loops with variable cutoff value until we get only two columns
def remove_tail_main(df, cut_val, sample_id):
  old_drop_cols = []
  drop_cols = remove_tail(df, cut_val, sample_id)
  while len(drop_cols) != 2:  
    # If len of columns to be dropped are 0 the select the 2 values smaller and bigger of the previous iteration
    if len(drop_cols) == 0 or len(drop_cols) == 1:
      drop_cols = [np.min(old_drop_cols), np.max(old_drop_cols)]
      break
    else:
      old_drop_cols = drop_cols
      drop_cols = remove_tail(df, cut_val, sample_id)
      cut_val += 2
  return drop_cols

In [8]:
def get_working_data_idx_v2(df_mean_sample, sample_id, cut_val = 0):

  df = df_mean_sample.iloc[[sample_id - 1]]

  #agg_df = aggreagator_v2(df) disable aggragator causing odd data points

  col_vals = remove_tail_main(df, cut_val=cut_val, sample_id=sample_id)

  left_index_no = df.columns.get_loc(col_vals[0])
  rigth_index_no = df.columns.get_loc(col_vals[1])

  return left_index_no, rigth_index_no

In [9]:
def cut_off_idx(df): # return tuple with cut-off values (index_left_side, index_rigth_side)
  df_mean_sample = df.groupby(['PBRS_id']).mean()
  df_mean_sample = df_mean_sample.drop(['row', 'Channels', 'Distance_km', 'power_dBm'], axis = 1)

  cut_points = {}
  print('Calculating the cutoff values:')
  for sample in tqdm(df_mean_sample.index):
    left_index_no, rigth_index_no = get_working_data_idx_v2(df_mean_sample, sample_id = sample)
    cut_points[sample] = (left_index_no, rigth_index_no)

  left_cut_off = np.min([v[0] for k, v in cut_points.items()])
  rigth_cut_off = np.max([v[1] for k, v in cut_points.items()])

  cut_point = (left_cut_off, rigth_cut_off)
  return cut_point

## Select working data

In [10]:
def select_working_data(df, cut_point): # implement cut_off values and Smoothing original df after cutt-of
  extra_info_df = df.iloc[:, 0:6]
  data_df = df.iloc[:, cut_point[0]+6:cut_point[1]+6]
  smoothed_data_frame = pd.DataFrame(savgol_filter(data_df, window_length = 5, polyorder = 2))
  
  # Adding back headers to the smoothed data
  rename_col = {}
  cnt = 0
  for col in data_df.columns:
    h_col = round(float(col), 5)
    rename_col[cnt] = h_col
    cnt += 1
  smoothed_data_frame.rename(columns=rename_col, inplace=True)

  working_df = pd.merge(extra_info_df, smoothed_data_frame, left_index=True, right_index=True)
  return working_df

In [11]:
def main_cutoff(input_path):
  print('Reading data from file.')
  df = pd.read_excel(input_path, sheet_name = "Sheet1", skiprows=1)
  df.iloc[0].fillna(method='bfill', inplace=True)
  df.iloc[1:df.shape[0]].fillna(method='pad', inplace=True)
  # Adding missing PBRS_id
  snippet = []
  val_cnt = 1
  idx = 0
  for sample in range(1, 101):
    for sample_id in range(0, 25):
      df.at[idx, 'PBRS_id'] = val_cnt
      idx +=1
      snippet.append(val_cnt)
    val_cnt += 1
  df['PBRS_id'] = df['PBRS_id'].astype(int)

  cut_point = cut_off_idx(df)
  working_df = select_working_data(df, cut_point)

  return working_df

## Run main cutoff

In [12]:
#input_path = path + "/Spectrum/16QAM_v2/Copy of dataSet_gamma0_Spectrum31MHz_Samples_16QAM_75GHz_LongHaul_input_25x81km_primeStep.xlsx"
#orking_df = main_cutoff(input_path)
#display(working_df)

In [13]:
#working_df.to_csv(path+"/Spectrum/NN/new_data_working_df.csv", index=False)

#New Train/Test Split

## Get Reference data

In [14]:
def get_reference_and_sample_data(working_df, num_reference):
  import random
  num_of_ids = len(working_df['PBRS_id'].unique())

  l = list(range(1, num_of_ids+1))
  random.shuffle(l)
  reference_arr = (l[:num_reference])
  sample_arr = l[-(num_of_ids-num_reference):]

  reference_data_df = working_df[working_df['PBRS_id'].isin(reference_arr)]
  sample_data_df = working_df[working_df['PBRS_id'].isin(sample_arr)]
  
  return reference_data_df, sample_data_df

## Get test/train split

In [15]:
def get_idx_train_test_split(working_df, trainingProp = 0.8):
  rows_training = np.array([])
  rows_testing = np.array([])
  distances = [x*80 for x in range(1, 26)]
  for d in distances:
    distance_working_df = working_df.loc[working_df['Distance_km'] == d]
    rows_mixed=np.random.permutation(distance_working_df.shape[0])

    training_amt = math.ceil(distance_working_df.shape[0]*trainingProp)
    testing_amt = distance_working_df.shape[0] - training_amt

    rows_training = np.append(rows_training, rows_mixed[:training_amt])
    rows_testing = np.append(rows_testing, rows_mixed[-testing_amt:])
  
  rows_training = rows_training.astype('int').tolist()
  rows_testing = rows_testing.astype('int').tolist()

  return rows_training, rows_testing

## Get frequencies filter

In [16]:
def frequencies_filter(df, num_selected_freq=20):
  info_df = df.iloc[:, 0:6]
  data_df = df.iloc[:, 6:df.shape[1]]
  span_val = int(data_df.shape[1]/num_selected_freq)
  for i in range(num_selected_freq):
    df2 = data_df.iloc[:, (i+1)*span_val].to_frame()
    if i+1 == 1:
      new_df = df2
    else:
      new_df = pd.merge(new_df, df2, left_index=True, right_index=True)
  new_df = info_df.join(new_df)
  
  return new_df

## Get features

In [17]:
#sample_freq_data_df
#distance_reference_df
#selected_rows = rows_training
def get_features(sample_freq_data_df, reference_freq_data_df, selected_rows, test_references_params):
  data_distance = []
  data_mean = []
  data_std = []
  data_pearson_min = []
  data_pearson_mean = []
  data_pearson_max = []
  data_CI_lower = []
  data_CI_upper = []

  if test_references_params[0] == True: # Test reference distance portion
    reference_freq_data_df = reference_freq_data_df['Distance_km'].isin(test_references_params[1])

  for row_idx in tqdm(selected_rows):
    reference_Y = reference_freq_data_df.iloc[row_idx,3] # distance of row selected

    data_Y = sample_freq_data_df.iloc[row_idx,3] # distance of row selected
    data_X = np.array(sample_freq_data_df.iloc[row_idx,6:sample_freq_data_df.shape[1]]) #array of data in row selected
    data_distance.append(data_Y)
    data_std.append(np.std(data_X))
    data_mean.append(np.mean(data_X))

    data_ci = st.t.interval(alpha=0.90, df=len(data_X)-1, loc=np.mean(data_X), scale=st.sem(data_X))
    data_confidence_int_lower = data_ci[0]
    data_confidence_int_upper = data_ci[1]

    # Get reference data -----------------
    distance_reference_df = reference_freq_data_df.loc[reference_freq_data_df['Distance_km'] == data_Y]
    if distance_reference_df.empty:
      distance_reference_df = reference_freq_data_df.copy()

    pearson_vals = []
    confidence_vals_lower = []
    confidence_vals_upper = []
    for reference_row in range(0, distance_reference_df.shape[0]):
      reference_X = np.array(distance_reference_df.iloc[reference_row,6:distance_reference_df.shape[1]])
      pearson_vals.append(round(pearsonr(data_X, reference_X)[0], 5))

      reference_ci = st.t.interval(alpha=0.90, df=len(reference_X)-1, loc=np.mean(reference_X), scale=st.sem(reference_X))
      reference_confidence_int_lower = reference_ci[0]
      confidence_vals_lower.append(round(abs(reference_confidence_int_lower - data_confidence_int_lower), 5))
      reference_confidence_int_upper = reference_ci[1]
      confidence_vals_upper.append(round(abs(reference_confidence_int_upper - data_confidence_int_upper),5))

    data_pearson_min.append(np.min(pearson_vals))
    data_pearson_mean.append(np.mean(pearson_vals))    
    data_pearson_max.append(np.max(pearson_vals))

    data_CI_lower.append(np.min(confidence_vals_lower))
    data_CI_upper.append(np.min(confidence_vals_upper))

  data = {'distance':data_distance,
                  'mean':data_mean,
                  'std':data_std,
                  'pearson_min':data_pearson_min,
                  'pearson_mean':data_pearson_mean,
                  'pearson_max':data_pearson_max,
                  'delta_CI_min':data_CI_lower,
                  'delta_CI_max':data_CI_upper}

  data_df = pd.DataFrame(data)

  
  return data_df

## Main train/test split

In [18]:
def get_train_test_split_main(input_path, num_of_freq_selected, reference_amt, test_references_params):
  working_df = pd.read_csv(input_path)
  working_df['Distance_km'] = working_df['Distance_km'].astype(int)
  #working_df = working_df.sample(frac = 1) # Shuffle df

  reference_data_df, sample_data_df = get_reference_and_sample_data(working_df, reference_amt)

  reference_freq_data_df = frequencies_filter(reference_data_df, num_selected_freq=num_of_freq_selected)
  sample_freq_data_df = frequencies_filter(sample_data_df, num_selected_freq=num_of_freq_selected)

  rows_training, rows_testing = get_idx_train_test_split(sample_data_df, trainingProp = 0.8)

  print('Calculating train split:')
  training_df = get_features(sample_freq_data_df, reference_freq_data_df, rows_training, test_references_params)
  print('Calculating test split:')
  testing_df = get_features(sample_freq_data_df, reference_freq_data_df, rows_testing, test_references_params)

  return training_df, testing_df, reference_freq_data_df, sample_freq_data_df, rows_training, rows_testing

# Bootstrapping

## Get max and min values of each column

In [ ]:
def get_max_min_val_col(working_df):
  distances = [x*80 for x in range(1, 26)]
  data_working_df = working_df.iloc[:, 6:working_df.shape[1]]
  h_list = list(data_working_df.columns.values)
  max_min_col_dict = {header:[] for header in h_list}

  for d in tqdm(distances):
    boot_dist_df = working_df.loc[working_df['Distance_km'] == d]
    for idx in range(6, boot_dist_df.shape[1]):
      max_val_col = float(boot_dist_df.iloc[:, [idx]].max()) # max val in col
      min_val_col = float(boot_dist_df.iloc[:, [idx]].min()) # min val in col
      h = boot_dist_df.iloc[:, [idx]].columns # header of col
      h = h[0]
      max_min_col_dict[h].append((d, min_val_col, max_val_col))
  return max_min_col_dict

## Create new samples using bootstrap technique

In [ ]:
def create_bootstrap_samples(working_df, max_min_col_dict, num_of_new_samples, previos_amt_of_samples=13):
  import random

  row_data = []
  PBRS_id_data = []
  Distance_km_data = []
  power_dBm_data = []
  span_data = []

  h_list = list(working_df.columns.values)
  new_data = {header:[] for header in h_list}

  last_row_in_old_df_val = working_df.shape[0]
  for new_sample in tqdm(range(num_of_new_samples)): # generating (num_of_new_samples) new samples for en data
    for d in distances:
      new_data['row'].append(last_row_in_old_df_val)
      last_row_in_old_df_val += 1
      new_data['PBRS_id'].append(new_sample+(previos_amt_of_samples+1))
      new_data['Channels'].append(1)

      new_data['Distance_km'].append(d)
      new_data['power_dBm'].append(0)
      new_data['#span'].append(int(d/80))

      for k, v in max_min_col_dict.items(): # header:(d, min_val_col, max_val_col)
        for val in v:
          if val[0] == d:
            bootstrap_val = round(random.uniform(val[1], val[2]), 14)
            new_data[str(k)].append(bootstrap_val)
            break
  new_data_df = pd.DataFrame.from_dict(new_data)

  return new_data_df

## Combine previos df with bootstraped df

In [ ]:
input_path = path + "/Spectrum/NN/new_data_working_df.csv"
working_df = pd.read_csv(input_path)
working_df['Distance_km'] = working_df['Distance_km'].astype(int)

max_min_col_dict = get_max_min_val_col(working_df)
new_data_df = create_bootstrap_samples(working_df, max_min_col_dict, num_of_new_samples=400, previos_amt_of_samples=100)
new_working_df = working_df.append(new_data_df)

new_working_df.to_csv(path + "/Spectrum/NN/bootstraped_new_data_working_df.csv", index=False)

100%|██████████| 400/400 [01:48<00:00,  3.70it/s]


#Test new data

## Hope better

In [24]:
def train_test_split_features(training_df, testing_df):
  Y = training_df.iloc[:, 0].to_numpy().reshape(-1,1)
  X = training_df.iloc[:, 1:(training_df.shape[1])]

  sc_input = MinMaxScaler()
  sc_output = MinMaxScaler()
  Y_train = sc_output.fit_transform(Y) # convert distances to values from 0 to 1
  X_train = sc_input.fit_transform(X) # convert features to values from 0 to 1

  Y = testing_df.iloc[:, 0].to_numpy().reshape(-1,1)
  X = testing_df.iloc[:, 1:(testing_df.shape[1])]

  Y_test = sc_output.fit_transform(Y) # convert distances to values from 0 to 1
  X_test = sc_input.fit_transform(X) # convert features to values from 0 to 1

  joblib.dump(sc_input, path+'/Spectrum/NN/train_test_splits/mod_scaler_input_30_10.joblib')
  joblib.dump(sc_output, path+'/Spectrum/NN/train_test_splits/mod_scaler_output_30_10.joblib')

  return Y_train, X_train, Y_test, X_test, sc_input, sc_output

In [25]:
def get_label_data_splits(input_path, num_of_freq_selected, reference_amt, split_done=False, test_references_params=(False, [80, 1040, 2000])):

  if split_done == False:
    training_df, testing_df, reference_freq_data_df, sample_freq_data_df, rows_training, rows_testing = get_train_test_split_main(input_path, num_of_freq_selected, reference_amt, test_references_params)

    training_df.to_csv(path+"/Spectrum/NN/train_test_splits/mod_bootstraped_new_data_training_data_NN_freq_"+str(num_of_freq_selected)+"_reference_"+str(reference_amt)+".csv", header=True, index=False)
    testing_df.to_csv(path+"/Spectrum/NN/train_test_splits/mod_bootstraped_new_data_testing_data_NN_freq_"+str(num_of_freq_selected)+"_reference_"+str(reference_amt)+".csv", header=True, index=False)

    reference_freq_data_df.to_csv(path+"/Spectrum/NN/train_test_splits/mod_bootstraped_new_data_Reference_NN_freq_"+str(num_of_freq_selected)+"_reference_"+str(reference_amt)+".csv", header=True, index=False)
    sample_freq_data_df.to_csv(path+"/Spectrum/NN/train_test_splits/mod_bootstraped_new_data_Sample_NN_freq_"+str(num_of_freq_selected)+"_reference_"+str(reference_amt)+".csv", header=True, index=False)

    pickle.dump(rows_training, open(path+"/Spectrum/NN/train_test_splits/mod_bootstraped_traininig_Rows_NN_freq_"+str(num_of_freq_selected)+"_reference_"+str(reference_amt)+".pkl","wb"))
    pickle.dump(rows_testing, open(path+"/Spectrum/NN/train_test_splits/mod_bootstraped_testing_Rows_NN_freq_"+str(num_of_freq_selected)+"_reference_"+str(reference_amt)+".pkl","wb"))
  else:
    training_df = pd.read_csv(path+"/Spectrum/NN/train_test_splits/mod_bootstraped_new_data_training_data_NN_freq_"+str(num_of_freq_selected)+"_reference_"+str(reference_amt)+".csv")
    testing_df = pd.read_csv(path+"/Spectrum/NN/train_test_splits/mod_bootstraped_new_data_testing_data_NN_freq_"+str(num_of_freq_selected)+"_reference_"+str(reference_amt)+".csv")
    
    reference_freq_data_df = pd.read_csv(path+"/Spectrum/NN/train_test_splits/mod_bootstraped_new_data_Reference_NN_freq_"+str(num_of_freq_selected)+"_reference_"+str(reference_amt)+".csv")
    sample_freq_data_df = pd.read_csv(path+"/Spectrum/NN/train_test_splits/mod_bootstraped_new_data_Sample_NN_freq_"+str(num_of_freq_selected)+"_reference_"+str(reference_amt)+".csv")
    with open(path+"/Spectrum/NN/train_test_splits/mod_bootstraped_traininig_Rows_NN_freq_"+str(num_of_freq_selected)+"_reference_"+str(reference_amt)+".pkl", 'rb') as f:
      rows_training = pickle.load(f)
    with open(path+"/Spectrum/NN/train_test_splits/mod_bootstraped_testing_Rows_NN_freq_"+str(num_of_freq_selected)+"_reference_"+str(reference_amt)+".pkl", 'rb') as f:
      rows_testing = pickle.load(f)  
  
  Y_train, X_train, Y_test, X_test, sc_input, sc_output = train_test_split_features(training_df, testing_df)

  return Y_train, X_train, Y_test, X_test, training_df, testing_df, reference_freq_data_df, sample_freq_data_df, rows_training, rows_testing, sc_input, sc_output

In [26]:
def predict_test(model_ann, Y_test, X_test, sc_output_c, testing_df, remove_outliers=0.05):
  print('Results (freq_'+str(num_of_freq_selected)+'_reference_'+str(reference_amt)+')')
  # Predit
  start_time = time.time()
  Y_test_pred=model_ann.predict(X_test)
  pred=list(Y_test_pred)

  print('Pred:')
  print(pred)
  
  time_eval_ann=time.time()-start_time

  dist_min=sc_output_c.data_min_[0] # 80
  dist_max=sc_output_c.data_max_[0] # 2000

  real=list(list(zip(*Y_test))[0])
  real_abs=np.add(dist_min,np.multiply((dist_max-dist_min),real)) # convert back to distances values from scalar
  real_abs=[int(np.round(i)) for i in real_abs]
  pred_abs=np.add(dist_min,np.multiply((dist_max-dist_min),pred)) # convert back to distances values from scalar

  print('Real_abs:')
  print(real_abs)
  print('Pred_abs:')
  print(pred_abs)

  real_abs_e1 = real_abs
  real_abs_e2 = real_abs

  pred_abs_e1 = pred_abs
  pred_abs_e2 = pred_abs

  error=np.divide(np.abs(np.subtract(np.array(real_abs),np.array(pred_abs))),np.array(real_abs))
  error_v2 = np.abs(np.subtract(np.array(real_abs),np.array(pred_abs)))

  remove_outliers = math.ceil((len(error_v2)*remove_outliers)/2)

  for outlier in range(1, remove_outliers+1):
    min_idx_e1 = np.argmin(error)
    max_idx_e1 = np.argmax(error)

    min_idx_e2 = np.argmin(error_v2)
    max_idx_e2 = np.argmax(error_v2)

    error = np.delete(error, min_idx_e1)
    error = np.delete(error, max_idx_e1)
    real_abs_e1 = np.delete(real_abs_e1, min_idx_e1)
    real_abs_e1 = np.delete(real_abs_e1, max_idx_e1)
    pred_abs_e1 = np.delete(pred_abs_e1, min_idx_e1)
    pred_abs_e1 = np.delete(pred_abs_e1, max_idx_e1)

    error_v2 = np.delete(error_v2, min_idx_e2)
    error_v2 = np.delete(error_v2, max_idx_e2)
    real_abs_e2 = np.delete(real_abs_e2, min_idx_e1)
    real_abs_e2 = np.delete(real_abs_e2, max_idx_e1)
    pred_abs_e2 = np.delete(pred_abs_e2, min_idx_e1)
    pred_abs_e2 = np.delete(pred_abs_e2, max_idx_e1)

  res=pd.DataFrame({"dist":real_abs_e1,"pred":pred_abs_e1,"error":error})
  res.to_csv(path+"/Spectrum/NN/NN_results_error/mod_bootstraped_new_data_results_curve_freq_"+str(num_of_freq_selected)+"_reference_"+str(reference_amt)+".csv", header=True, index=False)

  res_v2=pd.DataFrame({"dist":real_abs_e2,"pred":pred_abs_e2,"error":error_v2})
  res.to_csv(path+"/Spectrum/NN/NN_results_error/mod_bootstraped_new_data_results_deltas_freq_"+str(num_of_freq_selected)+"_reference_"+str(reference_amt)+".csv", header=True, index=False)
  
  plt.plot(real_abs_e1,error,'bo')
  plt.xlabel('Distances') 
  plt.ylabel('Error') 
  plt.title("Results Curve (freq_"+str(num_of_freq_selected)+"_reference_"+str(reference_amt)+")")
  plt.savefig(path+"/Spectrum/NN/NN_results_plots/mod_bootstraped_new_data_results_curve_freq_"+str(num_of_freq_selected)+"_reference_"+str(reference_amt)+".png")
  plt.show()
  
  plt.plot(real_abs_e2,error_v2,'bo')
  plt.xlabel('Distances') 
  plt.ylabel('Error') 
  plt.title("Results Deltas (freq_"+str(num_of_freq_selected)+"_reference_"+str(reference_amt)+")")
  plt.savefig(path+"/Spectrum/NN/NN_results_plots/bootstraped_new_data_results_deltas_freq_"+str(num_of_freq_selected)+"_reference_"+str(reference_amt)+".png")
  plt.show()

In [27]:
def train_model(X_train, Y_train):
  print('Training model (freq_'+str(num_of_freq_selected)+'_reference_'+str(reference_amt)+')...')
  from keras.models import Sequential
  from keras.layers import Dense,Conv2D, Flatten, Dropout
  from keras.wrappers.scikit_learn import KerasRegressor
  from keras.callbacks import EarlyStopping
  from tqdm.keras import TqdmCallback

  def ann():
      model = Sequential()
      model.add(Dense(76, input_dim = 7, activation = 'tanh'))
      model.add(Dense(38,activation='tanh'))      
      model.add(Dense(19,activation='tanh'))
      model.add(Dense(19,activation='tanh'))
      model.add(Dense(9,activation='tanh'))
      model.add(Dense(1))
      model.compile(loss ='mean_squared_error',optimizer = 'RMSprop')
      return model

  model_ann = KerasRegressor(build_fn=ann,epochs=8000,batch_size=32, verbose=0)

  start_time = time.time()
  callback = [EarlyStopping(monitor='loss', patience=500)]#, TqdmCallback(verbose=2)]
  model_ann.fit(X_train, Y_train, callbacks=callback)
  time_train_ann = time.time() - start_time

  model_ann.model.save(path+"/Spectrum/NN/NNs/mod_bootstraped_final_new_data_NN_model_freq_"+str(num_of_freq_selected)+"_reference_"+str(reference_amt)+".h5")
  model_ann.model.save_weights(path+"/Spectrum/NN/NNs/mod_bootstraped_final_new_data_NN_model_Weights_freq_"+str(num_of_freq_selected)+"_reference_"+str(reference_amt)+".h5", overwrite=True)
  
  return model_ann

In [28]:
num_of_freq_selected = 10
reference_amt = 30
input_path = path + "/Spectrum/NN/bootstraped_new_data_working_df.csv"

Y_train_c, X_train_c, Y_test_c, X_test_c, training_df_c, testing_df_c, reference_freq_data_df_c, sample_freq_data_df_c, rows_training_c, rows_testing_c, sc_input_c, sc_output_c = get_label_data_splits(input_path, num_of_freq_selected, reference_amt, split_done=False)

Calculating train split:


100%|██████████| 9400/9400 [05:39<00:00, 27.69it/s]


Calculating test split:


100%|██████████| 2350/2350 [01:25<00:00, 27.40it/s]


In [ ]:
model_ann_c = train_model(X_train_c, Y_train_c)
'''
# Load network
from keras.models import load_model
model_ann_c = KerasRegressor(build_fn=ann, epochs=8000, batch_size=32, verbose=0)
model_ann_c.model = load_model(path+"/Spectrum/NN/NNs/bootstraped_final_new_data_NN_model_freq_"+str(num_of_freq_selected)+"_reference_"+str(reference_amt)+".h5")
'''

Training model (freq_10_reference_30)...


In [ ]:
#predict_test(model_ann_c, Y_test_c, X_test_c, sc_output_c)
predict_test(model_ann_c, Y_test_c, X_test_c, sc_output_c, testing_df_c, remove_outliers=0.05)

# Short-cut

In [ ]:
def get_reference_and_sample_data(working_df, num_reference):
  import random
  num_of_ids = len(working_df['PBRS_id'].unique())

  l = list(range(1, num_of_ids+1))
  random.shuffle(l)
  reference_arr = (l[:num_reference])
  sample_arr = l[-(num_of_ids-num_reference):]

  reference_data_df = working_df[working_df['PBRS_id'].isin(reference_arr)]
  sample_data_df = working_df[working_df['PBRS_id'].isin(sample_arr)]
  
  return reference_data_df, sample_data_df

def frequencies_filter(df, num_selected_freq=20):
  info_df = df.iloc[:, 0:6]
  data_df = df.iloc[:, 6:df.shape[1]]
  span_val = int(data_df.shape[1]/num_selected_freq)
  for i in range(num_selected_freq):
    df2 = data_df.iloc[:, (i+1)*span_val].to_frame()
    if i+1 == 1:
      new_df = df2
    else:
      new_df = pd.merge(new_df, df2, left_index=True, right_index=True)
  new_df = info_df.join(new_df)
  
  return new_df

def get_idx_train_test_split(working_df, trainingProp = 0.8):
  rows_training = np.array([])
  rows_testing = np.array([])
  distances = [x*80 for x in range(1, 26)]
  for d in distances:
    distance_working_df = working_df.loc[working_df['Distance_km'] == d]
    rows_mixed=np.random.permutation(distance_working_df.shape[0])

    training_amt = math.ceil(distance_working_df.shape[0]*trainingProp)
    testing_amt = distance_working_df.shape[0] - training_amt

    rows_training = np.append(rows_training, rows_mixed[:training_amt])
    rows_testing = np.append(rows_testing, rows_mixed[-testing_amt:])
  
  rows_training = rows_training.astype('int').tolist()
  rows_testing = rows_testing.astype('int').tolist()

  return rows_training, rows_testing

# Load network
from keras.models import Sequential
from keras.layers import Dense,Conv2D, Flatten
from keras.wrappers.scikit_learn import KerasRegressor
from keras.callbacks import EarlyStopping
from tqdm.keras import TqdmCallback

def ann():
    model = Sequential()
    model.add(Dense(76, input_dim = 7, activation = 'tanh'))
    model.add(Dense(38,activation='tanh'))
    model.add(Dense(38,activation='tanh'))
    model.add(Dense(19,activation='tanh'))
    model.add(Dense(19,activation='tanh'))
    model.add(Dense(9,activation='tanh'))
    model.add(Dense(9,activation='tanh'))
    model.add(Dense(1))
    model.compile(loss ='mean_squared_error',optimizer = 'RMSprop')
    return model

def get_feature(data_split_df, reference_freq_data_df, reference_distance_eval_params=(False, 80)):
  reference_eval = reference_freq_data_df.copy()
  if reference_distance_eval_params[0] == True:
    reference_eval = reference_freq_data_df.loc[reference_freq_data_df['Distance_km'] == reference_distance_eval_params[1]]

  data_distance = []
  data_mean = []
  data_std = []
  data_pearson_min = []
  data_pearson_mean = []
  data_pearson_max = []
  data_CI_lower = []
  data_CI_upper = []

  for idx in tqdm(range(len(data_split_df))):
    Y_label = data_split_df.iloc[idx, 3]
    X_label = data_split_df.iloc[idx, 6:data_split_df.shape[1]]
    data_distance.append(int(Y_label))
    data_std.append(round(np.std(X_label), 5))
    data_mean.append(round(np.mean(X_label), 5))
    
    data_ci = st.t.interval(alpha=0.90, df=len(X_label)-1, loc=np.mean(X_label), scale=st.sem(X_label))
    data_confidence_int_lower = data_ci[0]
    data_confidence_int_upper = data_ci[1]

    pearson_vals = []
    ci_lower_delta = []
    ci_upper_delta = []
    for reference_idx in range(len(reference_eval)):
      X_reference = reference_eval.iloc[reference_idx, 6:reference_eval.shape[1]]
      pearson_vals.append(round(pearsonr(X_label, X_reference)[0], 5))

      reference_ci = st.t.interval(alpha=0.90, df=len(X_reference)-1, loc=np.mean(X_reference), scale=st.sem(X_reference))
      reference_confidence_int_lower = reference_ci[0]
      reference_confidence_int_upper = reference_ci[1]

      delta_ci_lower = abs(reference_confidence_int_lower - data_confidence_int_lower)
      ci_lower_delta.append(round(delta_ci_lower, 5))
      delta_ci_upper = abs(reference_confidence_int_upper - data_confidence_int_upper)
      ci_upper_delta.append(round(delta_ci_upper, 5))

    data_pearson_min.append(np.min(pearson_vals))
    data_pearson_mean.append(np.mean(pearson_vals))    
    data_pearson_max.append(np.max(pearson_vals))

    data_CI_lower.append(np.min(ci_lower_delta))
    data_CI_upper.append(np.min(ci_upper_delta))

  data = {'distance':data_distance,
                  'mean':data_mean,
                  'std':data_std,
                  'pearson_min':data_pearson_min,
                  'pearson_mean':data_pearson_mean,
                  'pearson_max':data_pearson_max,
                  'delta_CI_min':data_CI_lower,
                  'delta_CI_max':data_CI_upper}

  data_df = pd.DataFrame(data)
  return data_df

In [ ]:
train_df = pd.read_csv(path+"/Spectrum/NN/train_test_splits/mod_bootstraped_new_data_training_data_NN_freq_10_reference_30.csv")
test_df = pd.read_csv(path+"/Spectrum/NN/train_test_splits/mod_bootstraped_new_data_testing_data_NN_freq_10_reference_30.csv")

reference_freq_data_df = pd.read_csv(path+"/Spectrum/NN/train_test_splits/mod_bootstraped_new_data_Reference_NN_freq_10_reference_30.csv")
sample_freq_data_df = pd.read_csv(path+"/Spectrum/NN/train_test_splits/mod_bootstraped_new_data_Sample_NN_freq_10_reference_30.csv")

with open(path+"/Spectrum/NN/train_test_splits/mod_bootstraped_traininig_Rows_NN_freq_10_reference_30.pkl", 'rb') as f:
  rows_training = pickle.load(f)
with open(path+"/Spectrum/NN/train_test_splits/mod_bootstraped_testing_Rows_NN_freq_10_reference_30.pkl", 'rb') as f:
  rows_testing = pickle.load(f) 

In [ ]:
num_of_freq_selected = 10
reference_amt = 30

# Load network
from keras.models import load_model
mm = KerasRegressor(build_fn=ann, epochs=8000, batch_size=32, verbose=0)
mm.model = load_model(path+"/Spectrum/NN/NNs/mod_bootstraped_final_new_data_NN_model_freq_10_reference_30.h5")

res = {}
reference_test = [80, 1040, 2000]
distances = [x*80 for x in range(1, 26)]

#Y = train_df.iloc[:, 0].to_numpy().reshape(-1,1)
#X = train_df.iloc[:, 1:train_df.shape[1]-2]

#sc_input = MinMaxScaler()
#sc_output = MinMaxScaler()
#Y_train = sc_output.fit_transform(Y) # convert distances to values from 0 to 1
#X_train = sc_input.fit_transform(X) # convert features to values from 0 to 1

for r in reference_test:
  test_split = sample_freq_data_df.iloc[rows_testing]
  test_df = get_feature(test_split, reference_freq_data_df, reference_distance_eval_params=(True, r))
  
  dist_min=sc_output_c.data_min_[0] # 80
  dist_max=sc_output_c.data_max_[0] # 2000

  Y = test_df.iloc[:, 0].to_numpy().reshape(-1,1)
  X = test_df.iloc[:, 1:test_df.shape[1]]

  Y_test = sc_output_c.fit_transform(Y) # convert distances to values from 0 to 1
  X_test = sc_input_c.fit_transform(X) # convert features to values from 0 to 1

  # Prediction
  Y_test_pred=mm.predict(X_test)
  pred=list(Y_test_pred)

  real=list(list(zip(*Y_test))[0])
  real_abs=np.add(dist_min,np.multiply((dist_max-dist_min),real)) # convert back to distances values from scalar
  real_abs=[int(np.round(i)) for i in real_abs]
  pred_abs=np.add(dist_min,np.multiply((dist_max-dist_min),pred)) # convert back to distances values from scalar

  real_split ={}
  for d in distances:
    matches = [i for i in range(0,len(real_abs)) if real_abs[i]==d]
    real_split[d] = matches
  
  pred_res = []
  pred_m = []
  for d_real, matches_idx in real_split.items():
    pred_m = [pred_abs[i] for i in matches_idx]
    Q1 = np.percentile(pred_m, 25,interpolation = 'midpoint')
    Q3 = np.percentile(pred_m, 75,interpolation = 'midpoint')

    IQR = Q3 - Q1
    upper = Q3+1.5*IQR
    lower = Q1-1.5*IQR
    filtered_pred_m = []
    for v in pred_m:
      if v <= upper and v >= lower:
        filtered_pred_m.append(v)
    pred_res.append(round(np.mean(filtered_pred_m), 5))
  
  res[r] = pred_res
print('-----------------------------------------------------------------------')
print('Resuslts')

plt.figure(figsize=(15, 10))
plt.plot(distances, distances,  linestyle='--')
for re in reference_test:
  plt.plot(distances, res[re], label='Reference '+str(re))

plt.title('Real vs Predicted Distances')
plt.xlabel('Real Distances')
plt.ylabel('Predicted Distances')
plt.xlim(np.min(distances)-25, np.max(distances)+25)
plt.ylim(np.min(distances)-25, np.max(distances)+25)
plt.yticks(distances)
plt.xticks(distances)
plt.legend(loc="best")
plt.show()

for re in reference_test:
  print(res[re])

print('-----------------------------------------------------------------------')

In [ ]:
num_of_freq_selected = 10
reference_amt = 30

# Load network
from keras.models import load_model
mm = KerasRegressor(build_fn=ann, epochs=8000, batch_size=32, verbose=0)
mm.model = load_model(path+"/Spectrum/NN/NNs/mod_bootstraped_final_new_data_NN_model_freq_10_reference_30.h5")

res = {}
reference_test = [x*80 for x in range(1, 26)]
distances = [x*80 for x in range(1, 26)]

#Y = train_df.iloc[:, 0].to_numpy().reshape(-1,1)
#X = train_df.iloc[:, 1:train_df.shape[1]-2]

#sc_input = MinMaxScaler()
#sc_output = MinMaxScaler()
#Y_train = sc_output.fit_transform(Y) # convert distances to values from 0 to 1
#X_train = sc_input.fit_transform(X) # convert features to values from 0 to 1

sc_input_c, sc_output_c

for r in reference_test:
  test_split = sample_freq_data_df.iloc[rows_testing]
  test_df = get_feature(test_split, reference_freq_data_df, reference_distance_eval_params=(True, r))
  
  dist_min=sc_output_c.data_min_[0] # 80
  dist_max=sc_output_c.data_max_[0] # 2000

  Y = test_df.iloc[:, 0].to_numpy().reshape(-1,1)
  X = test_df.iloc[:, 1:test_df.shape[1]]

  Y_test = sc_output_c.fit_transform(Y) # convert distances to values from 0 to 1
  X_test = sc_input_c.fit_transform(X) # convert features to values from 0 to 1

  # Prediction
  Y_test_pred=mm.predict(X_test)
  pred=list(Y_test_pred)

  
  real=list(list(zip(*Y_test))[0])
  real_abs=np.add(dist_min,np.multiply((dist_max-dist_min),real)) # convert back to distances values from scalar
  real_abs=[int(np.round(i)) for i in real_abs]
  pred_abs=np.add(dist_min,np.multiply((dist_max-dist_min),pred)) # convert back to distances values from scalar

  real_split ={}
  for d in distances:
    matches = [i for i in range(0,len(real_abs)) if real_abs[i]==d]
    real_split[d] = matches
  
  pred_res = []
  pred_m = []
  for d_real, matches_idx in real_split.items():
    pred_m = [pred_abs[i] for i in matches_idx]
    Q1 = np.percentile(pred_m, 25,interpolation = 'midpoint')
    Q3 = np.percentile(pred_m, 75,interpolation = 'midpoint')

    IQR = Q3 - Q1
    upper = Q3+1.5*IQR
    lower = Q1-1.5*IQR
    filtered_pred_m = []
    for v in pred_m:
      if v <= upper and v >= lower:
        filtered_pred_m.append(v)
    pred_res.append(round(np.mean(filtered_pred_m), 5))
  
  res[r] = pred_res
print('-----------------------------------------------------------------------')
print('Resuslts')

plt.figure(figsize=(15, 10))
plt.plot(distances, distances,  linestyle='--')
for re in reference_test:
  plt.plot(distances, res[re], label='Reference '+str(re))

plt.title('Real vs Predicted Distances')
plt.xlabel('Real Distances')
plt.ylabel('Predicted Distances')
plt.xlim(np.min(distances)-25, np.max(distances)+25)
plt.ylim(np.min(distances)-25, np.max(distances)+25)
plt.yticks(distances)
plt.xticks(distances)
plt.legend(loc="best")
plt.show()

for re in reference_test:
  print(res[re])

print('-----------------------------------------------------------------------')

# Test only pearson

In [ ]:
# Load network
from keras.models import Sequential
from keras.layers import Dense,Conv2D, Flatten
from keras.wrappers.scikit_learn import KerasRegressor
from keras.callbacks import EarlyStopping
from tqdm.keras import TqdmCallback

def ann():
  model = Sequential()
  model.add(Dense(76, input_dim = 3, activation = 'tanh'))
  model.add(Dense(38,activation='tanh'))
  model.add(Dense(19,activation='tanh'))
  model.add(Dense(1))
  model.compile(loss ='mean_squared_error',optimizer = 'RMSprop')
  return model

In [ ]:
def train_model(X_train, Y_train):
  print('Training model (freq_'+str(num_of_freq_selected)+'_reference_'+str(reference_amt)+')...')
  from keras.models import Sequential
  from keras.layers import Dense,Conv2D, Flatten
  from keras.wrappers.scikit_learn import KerasRegressor
  from keras.callbacks import EarlyStopping
  from tqdm.keras import TqdmCallback

  def ann():
      model = Sequential()
      model.add(Dense(76, input_dim = 3, activation = 'tanh'))
      model.add(Dense(38,activation='tanh'))
      model.add(Dense(19,activation='tanh'))
      model.add(Dense(1))
      model.compile(loss ='mean_squared_error',optimizer = 'RMSprop')
      return model

  model_ann = KerasRegressor(build_fn=ann,epochs=5000,batch_size=32, verbose=0)

  start_time = time.time()
  callback = [EarlyStopping(monitor='loss', patience=500)]#, TqdmCallback(verbose=2)]
  model_ann.fit(X_train, Y_train, callbacks=callback)
  time_train_ann = time.time() - start_time

  # Save model
  #pickle.dump(model_ann,open(path+"/Spectrum/NN/NNs/NN_model_freq_"+str(num_of_freq_selected)+"_reference_"+str(reference_amt)+".pkl","wb"))

  if os.path.isfile(path+"/Spectrum/NN/NNs/pearson_new_data_NN_model_freq_"+str(num_of_freq_selected)+"_reference_"+str(reference_amt)+".h5") is False:
    model_ann.model.save(path+"/Spectrum/NN/NNs/pearson_new_data_NN_model_freq_"+str(num_of_freq_selected)+"_reference_"+str(reference_amt)+".h5")
  
  return model_ann

In [ ]:
def predict_test(model_ann, Y_test, X_test, sc_output, remove_outliers=0.05):
  print('Results (freq_'+str(num_of_freq_selected)+'_reference_'+str(reference_amt)+')')
  # Predit
  start_time = time.time()
  Y_test_pred=model_ann.predict(X_test)
  pred=list(Y_test_pred)

  time_eval_ann=time.time()-start_time

  dist_min=sc_output.data_min_[0] # 80
  dist_max=sc_output.data_max_[0] # 2000

  real=list(list(zip(*Y_test))[0])
  real_abs=np.add(dist_min,np.multiply((dist_max-dist_min),real)) # convert back to distances values from scalar
  real_abs=[int(np.round(i)) for i in real_abs]
  pred_abs=np.add(dist_min,np.multiply((dist_max-dist_min),pred)) # convert back to distances values from scalar
  real_abs_e1 = real_abs
  real_abs_e2 = real_abs

  pred_abs_e1 = pred_abs
  pred_abs_e2 = pred_abs

  error=np.divide(np.abs(np.subtract(np.array(real_abs),np.array(pred_abs))),np.array(real_abs))
  error_v2 = np.abs(np.subtract(np.array(real_abs),np.array(pred_abs)))

  remove_outliers = math.ceil((len(error_v2)*remove_outliers)/2)

  for outlier in range(1, remove_outliers+1):
    min_idx_e1 = np.argmin(error)
    max_idx_e1 = np.argmax(error)

    min_idx_e2 = np.argmin(error_v2)
    max_idx_e2 = np.argmax(error_v2)

    error = np.delete(error, min_idx_e1)
    error = np.delete(error, max_idx_e1)
    real_abs_e1 = np.delete(real_abs_e1, min_idx_e1)
    real_abs_e1 = np.delete(real_abs_e1, max_idx_e1)
    pred_abs_e1 = np.delete(pred_abs_e1, min_idx_e1)
    pred_abs_e1 = np.delete(pred_abs_e1, max_idx_e1)

    error_v2 = np.delete(error_v2, min_idx_e2)
    error_v2 = np.delete(error_v2, max_idx_e2)
    real_abs_e2 = np.delete(real_abs_e2, min_idx_e1)
    real_abs_e2 = np.delete(real_abs_e2, max_idx_e1)
    pred_abs_e2 = np.delete(pred_abs_e2, min_idx_e1)
    pred_abs_e2 = np.delete(pred_abs_e2, max_idx_e1)

  res=pd.DataFrame({"dist":real_abs_e1,"pred":pred_abs_e1,"error":error})
  res.to_csv(path+"/Spectrum/NN/NN_results_error/new_data_pearson_results_curve_freq_"+str(num_of_freq_selected)+"_reference_"+str(reference_amt)+".csv", header=True, index=False)

  res_v2=pd.DataFrame({"dist":real_abs_e2,"pred":pred_abs_e2,"error":error_v2})
  res.to_csv(path+"/Spectrum/NN/NN_results_error/new_data_pearson_results_deltas_freq_"+str(num_of_freq_selected)+"_reference_"+str(reference_amt)+".csv", header=True, index=False)
  
  plt.plot(real_abs_e1,error,'bo')
  plt.xlabel('Distances') 
  plt.ylabel('Error') 
  plt.title("Results Curve (freq_"+str(num_of_freq_selected)+"_reference_"+str(reference_amt)+")")
  plt.savefig(path+"/Spectrum/NN/NN_results_plots/new_data_pearson_results_curve_freq_"+str(num_of_freq_selected)+"_reference_"+str(reference_amt)+".png")
  plt.show()
  
  plt.plot(real_abs_e2,error_v2,'bo')
  plt.xlabel('Distances') 
  plt.ylabel('Error') 
  plt.title("Results Deltas (freq_"+str(num_of_freq_selected)+"_reference_"+str(reference_amt)+")")
  plt.savefig(path+"/Spectrum/NN/NN_results_plots/new_data_pearson_results_deltas_freq_"+str(num_of_freq_selected)+"_reference_"+str(reference_amt)+".png")
  plt.show()


In [ ]:
num_of_freq_selected = 10
reference_amt = 30
training_input_path = path + "/Spectrum/NN/train_test_splits/new_data_training_data_NN_freq_10_reference_30.csv"
testing_input_path = path + "/Spectrum/NN/train_test_splits/new_data_testing_data_NN_freq_10_reference_30.csv"

training_df = pd.read_csv(training_input_path)
testing_df = pd.read_csv(testing_input_path)

Y = training_df.iloc[:, 0].to_numpy().reshape(-1,1)
X = training_df.iloc[:, 3:6]

sc_input = MinMaxScaler()
sc_output = MinMaxScaler()
Y_train_p = sc_output.fit_transform(Y) # convert distances to values from 0 to 1
X_train_p = sc_input.fit_transform(X) # convert features to values from 0 to 1

Y = testing_df.iloc[:, 0].to_numpy().reshape(-1,1)
X = testing_df.iloc[:, 3:6]

sc_input = MinMaxScaler()
sc_output = MinMaxScaler()
Y_test_p = sc_output.fit_transform(Y) # convert distances to values from 0 to 1
X_test_p = sc_input.fit_transform(X) # convert features to values from 0 to 1

In [ ]:
model_ann_p = train_model(X_train_p, Y_train_p)

In [ ]:
predict_test(model_ann_p, Y_test_p, X_test_p, sc_output)

## Test only stats

In [ ]:
# Load network
from keras.models import Sequential
from keras.layers import Dense,Conv2D, Flatten
from keras.wrappers.scikit_learn import KerasRegressor
from keras.callbacks import EarlyStopping
from tqdm.keras import TqdmCallback

def ann():
  model = Sequential()
  model.add(Dense(76, input_dim = 3, activation = 'tanh'))
  model.add(Dense(38,activation='tanh'))
  model.add(Dense(19,activation='tanh'))
  model.add(Dense(1))
  model.compile(loss ='mean_squared_error',optimizer = 'RMSprop')
  return model

In [ ]:
# Load network
from keras.models import Sequential
from keras.layers import Dense,Conv2D, Flatten
from keras.wrappers.scikit_learn import KerasRegressor
from keras.callbacks import EarlyStopping
from tqdm.keras import TqdmCallback

def ann():
  model = Sequential()
  model.add(Dense(76, input_dim = 4, activation = 'tanh'))
  model.add(Dense(38,activation='tanh'))
  model.add(Dense(19,activation='tanh'))
  model.add(Dense(1))
  model.compile(loss ='mean_squared_error',optimizer = 'RMSprop')
  return model

In [ ]:
def train_model(X_train, Y_train):
  print('Training model (freq_'+str(num_of_freq_selected)+'_reference_'+str(reference_amt)+')...')
  from keras.models import Sequential
  from keras.layers import Dense,Conv2D, Flatten
  from keras.wrappers.scikit_learn import KerasRegressor
  from keras.callbacks import EarlyStopping
  from tqdm.keras import TqdmCallback

  def ann():
      model = Sequential()
      model.add(Dense(76, input_dim = 4, activation = 'tanh'))
      model.add(Dense(38,activation='tanh'))
      model.add(Dense(19,activation='tanh'))
      model.add(Dense(1))
      model.compile(loss ='mean_squared_error',optimizer = 'RMSprop')
      return model

  model_ann = KerasRegressor(build_fn=ann,epochs=5000,batch_size=32, verbose=0)

  start_time = time.time()
  callback = [EarlyStopping(monitor='loss', patience=500)]#, TqdmCallback(verbose=2)]
  model_ann.fit(X_train, Y_train, callbacks=callback)
  time_train_ann = time.time() - start_time

  # Save model
  #pickle.dump(model_ann,open(path+"/Spectrum/NN/NNs/NN_model_freq_"+str(num_of_freq_selected)+"_reference_"+str(reference_amt)+".pkl","wb"))

  if os.path.isfile(path+"/Spectrum/NN/NNs/stats_new_data_NN_model_freq_"+str(num_of_freq_selected)+"_reference_"+str(reference_amt)+".h5") is False:
    model_ann.model.save(path+"/Spectrum/NN/NNs/stats_new_data_NN_model_freq_"+str(num_of_freq_selected)+"_reference_"+str(reference_amt)+".h5")
  
  return model_ann

In [ ]:
def predict_test(model_ann, Y_test, X_test, sc_output, remove_outliers=0.05):
  print('Results (freq_'+str(num_of_freq_selected)+'_reference_'+str(reference_amt)+')')
  # Predit
  start_time = time.time()
  Y_test_pred=model_ann.predict(X_test)
  pred=list(Y_test_pred)

  time_eval_ann=time.time()-start_time

  dist_min=sc_output.data_min_[0] # 80
  dist_max=sc_output.data_max_[0] # 2000

  real=list(list(zip(*Y_test))[0])
  real_abs=np.add(dist_min,np.multiply((dist_max-dist_min),real)) # convert back to distances values from scalar
  real_abs=[int(np.round(i)) for i in real_abs]
  pred_abs=np.add(dist_min,np.multiply((dist_max-dist_min),pred)) # convert back to distances values from scalar

  real_abs_e1 = real_abs
  real_abs_e2 = real_abs

  pred_abs_e1 = pred_abs
  pred_abs_e2 = pred_abs

  error=np.divide(np.abs(np.subtract(np.array(real_abs),np.array(pred_abs))),np.array(real_abs))
  error_v2 = np.abs(np.subtract(np.array(real_abs),np.array(pred_abs)))

  remove_outliers = math.ceil((len(error_v2)*remove_outliers)/2)

  for outlier in range(1, remove_outliers+1):
    min_idx_e1 = np.argmin(error)
    max_idx_e1 = np.argmax(error)

    min_idx_e2 = np.argmin(error_v2)
    max_idx_e2 = np.argmax(error_v2)

    error = np.delete(error, min_idx_e1)
    error = np.delete(error, max_idx_e1)
    real_abs_e1 = np.delete(real_abs_e1, min_idx_e1)
    real_abs_e1 = np.delete(real_abs_e1, max_idx_e1)
    pred_abs_e1 = np.delete(pred_abs_e1, min_idx_e1)
    pred_abs_e1 = np.delete(pred_abs_e1, max_idx_e1)

    error_v2 = np.delete(error_v2, min_idx_e2)
    error_v2 = np.delete(error_v2, max_idx_e2)
    real_abs_e2 = np.delete(real_abs_e2, min_idx_e1)
    real_abs_e2 = np.delete(real_abs_e2, max_idx_e1)
    pred_abs_e2 = np.delete(pred_abs_e2, min_idx_e1)
    pred_abs_e2 = np.delete(pred_abs_e2, max_idx_e1)

  res=pd.DataFrame({"dist":real_abs_e1,"pred":pred_abs_e1,"error":error})
  res.to_csv(path+"/Spectrum/NN/NN_results_error/new_data_stats_results_curve_freq_"+str(num_of_freq_selected)+"_reference_"+str(reference_amt)+".csv", header=True, index=False)

  res_v2=pd.DataFrame({"dist":real_abs_e2,"pred":pred_abs_e2,"error":error_v2})
  res.to_csv(path+"/Spectrum/NN/NN_results_error/new_data_stats_results_deltas_freq_"+str(num_of_freq_selected)+"_reference_"+str(reference_amt)+".csv", header=True, index=False)
  
  plt.plot(real_abs_e1,error,'bo')
  plt.xlabel('Distances') 
  plt.ylabel('Error') 
  plt.title("Results Curve (freq_"+str(num_of_freq_selected)+"_reference_"+str(reference_amt)+")")
  plt.savefig(path+"/Spectrum/NN/NN_results_plots/new_data_stats_results_curve_freq_"+str(num_of_freq_selected)+"_reference_"+str(reference_amt)+".png")
  plt.show()
  
  plt.plot(real_abs_e2,error_v2,'bo')
  plt.xlabel('Distances') 
  plt.ylabel('Error') 
  plt.title("Results Deltas (freq_"+str(num_of_freq_selected)+"_reference_"+str(reference_amt)+")")
  plt.savefig(path+"/Spectrum/NN/NN_results_plots/new_data_stats_results_deltas_freq_"+str(num_of_freq_selected)+"_reference_"+str(reference_amt)+".png")
  plt.show()

In [ ]:
num_of_freq_selected = 10
reference_amt = 30
training_input_path = path + "/Spectrum/NN/train_test_splits/new_data_training_data_NN_freq_10_reference_30.csv"
testing_input_path = path + "/Spectrum/NN/train_test_splits/new_data_testing_data_NN_freq_10_reference_30.csv"

training_df = pd.read_csv(training_input_path)
testing_df = pd.read_csv(testing_input_path)

Y = training_df.iloc[:, 0].to_numpy().reshape(-1,1)
X = training_df.iloc[:, [1, 2, 5, 6]]

sc_input = MinMaxScaler()
sc_output = MinMaxScaler()
Y_train_s = sc_output.fit_transform(Y) # convert distances to values from 0 to 1
X_train_s = sc_input.fit_transform(X) # convert features to values from 0 to 1

Y = testing_df.iloc[:, 0].to_numpy().reshape(-1,1)
X = testing_df.iloc[:, [1, 2, 5, 6]]


sc_input = MinMaxScaler()
sc_output = MinMaxScaler()
Y_test_s = sc_output.fit_transform(Y) # convert distances to values from 0 to 1
X_test_s = sc_input.fit_transform(X) # convert features to values from 0 to 1

In [ ]:
model_ann_s = train_model(X_train_s, Y_train_s)

In [ ]:
predict_test(model_ann_s, Y_test_s, X_test_s, sc_output)